In [1]:
import os
import zarr
import timm
import random
import json
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict
import sys
import torch

# import torchvision.transforms.functional as F
import random

warnings.filterwarnings("ignore")
sys.path.append("./src/")

from src.config import CFG
from src.dataloader import (
    read_zarr,
    read_info_json,
    scale_coordinates,
    create_dataset,
    create_segmentation_map,
    EziiDataset,
    drop_padding,
)
from src.network import Unet3D
from src.utils import save_images, PadToSize
from src.metric import (
    score,
    create_cls_pos,
    create_cls_pos_sikii,
    create_df,
    SegmentationLoss,
    DiceLoss,
)
from metric import visualize_epoch_results
from src.utils import save_images
from src.metric import score, create_cls_pos, create_cls_pos_sikii, create_df
from src.inference import inference, inference2pos

sample_submission = pd.read_csv("../../inputs/sample_submission.csv")

In [2]:
# padf = PadToSize(CFG.resolution)


# def last_padding(tomogram, slice_size):
#     # tomogram: (tensor)
#     b, d, h, w = tomogram.shape
#     last_padding = slice_size - d % slice_size
#     if last_padding == slice_size:
#         return tomogram
#     else:
#         return torch.cat(
#             [tomogram, torch.zeros(b, last_padding, h, w).to(tomogram.device)], dim=1
#         )


# def preprocess_tensor(tensor):
#     batch_size, depth, height, width = tensor.shape
#     tensor = tensor.unsqueeze(2)  # (b, d, h, w) -> (b, d, 1, h, w)
#     return tensor


# def inference(model, exp_name, train=True):
#     dataset = EziiDataset(
#         exp_names=[exp_name],
#         base_dir="../../inputs/train/",
#         particles_name=CFG.particles_name,
#         resolution=CFG.resolution,
#         zarr_type=["denoised"],
#         train=train,
#         slice=False,
#     )
#     res_array = CFG.original_img_shape[CFG.resolution]
#     pred_array = np.zeros(
#         (len(CFG.particles_name) + 1, res_array[0], res_array[1], res_array[2])
#     )
#     loader = DataLoader(dataset, batch_size=1, shuffle=False, pin_memory=True)
#     model.eval()
#     # tq = tqdm(loader)
#     for data in loader:  # 実験データ1つを取り出す
#         for i in range(0, data["normalized_tomogram"].shape[1], CFG.slice_):
#             normalized_tomogram = data["normalized_tomogram"][:, i : i + CFG.slice_]
#             normalized_tomogram = last_padding(normalized_tomogram, CFG.slice_)
#             normalized_tomogram = padf(normalized_tomogram)
#             normalized_tomogram = preprocess_tensor(normalized_tomogram).to("cuda")
#             pred = model(normalized_tomogram)
#             prob_pred = (
#                 torch.softmax(pred, dim=1).detach().cpu().numpy()
#             )  # torch.Size([1, 7, 32, 320, 320])
#             range_ = min(i + CFG.slice_, res_array[0])
#             hw_pad_diff = prob_pred.shape[-1] - res_array[-1]

#             if i >= res_array[0]:
#                 continue

#             if range_ == res_array[0]:
#                 pred_array[:, i:range_] += prob_pred[
#                     0, :, : res_array[0] - i, :-hw_pad_diff, :-hw_pad_diff
#                 ]
#             else:
#                 pred_array[:, i:range_] += prob_pred[
#                     0, :, :range_, :-hw_pad_diff, :-hw_pad_diff
#                 ]

#         if train:
#             segmentation_map = data["segmentation_map"]
#         else:
#             segmentation_map = None

#         normalized_tomogram = data["normalized_tomogram"]
#     # tq.close()

#     return pred_array, normalized_tomogram, segmentation_map  # (7, 92, 315, 315)


# def inference2pos(pred_segmask, exp_name):
#     import cc3d

#     cls_pos = []
#     Ascale_pos = []
#     res2ratio = CFG.resolution2ratio

#     for pred_cls in range(1, len(CFG.particles_name) + 1):
#         print(pred_cls, CFG.cls2particles[pred_cls])
#         cc, P = cc3d.connected_components(pred_segmask == pred_cls, return_N=True)
#         stats = cc3d.statistics(cc)

#         for z, y, x in stats["centroids"]:
#             Ascale_z = z * res2ratio[CFG.resolution] / res2ratio["A"]
#             Ascale_x = x * res2ratio[CFG.resolution] / res2ratio["A"]
#             Ascale_y = y * res2ratio[CFG.resolution] / res2ratio["A"]

#             cls_pos.append([pred_cls, z, y, x])
#             Ascale_pos.append([pred_cls, Ascale_z, Ascale_y, Ascale_x])

#     pred_original_df = create_df(Ascale_pos, exp_name)

#     return pred_original_df

In [3]:
train_dataset = EziiDataset(
    exp_names=CFG.train_exp_names,
    base_dir="../../inputs/train/",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.train_zarr_types,
    train=True,
    augmentation=False,
    slice=False,
    pre_read=True,
)

# train_nshuffle_dataset = EziiDataset(
#     exp_names=CFG.train_exp_names,
#     base_dir="../../inputs/train/",
#     particles_name=CFG.particles_name,
#     resolution=CFG.resolution,
#     zarr_type=CFG.train_zarr_types,
#     augmentation=False,
#     train=True,
# )

valid_dataset = EziiDataset(
    exp_names=CFG.valid_exp_names,
    base_dir="../../inputs/train/",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.valid_zarr_types,
    augmentation=False,
    train=True,
    slice=False,
    pre_read=True,
)

from tqdm import tqdm

train_loader = DataLoader(
    train_dataset,
    batch_size=CFG.batch_size,
    shuffle=True,
    drop_last=True,
    pin_memory=True,
    num_workers=CFG.num_workers,
)
# train_nshuffle_loader = DataLoader(
#     train_nshuffle_dataset,
#     batch_size=1,
#     shuffle=True,
#     drop_last=True,
#     pin_memory=True,
#     num_workers=CFG.num_workers,
# )
valid_loader = DataLoader(
    valid_dataset,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    num_workers=CFG.num_workers,
)

for data in tqdm(train_loader):
    normalized_tomogram = data["normalized_tomogram"]
    segmentation_map = data["segmentation_map"]
    break

normalized_tomogram.shape

  0%|          | 0/12 [00:05<?, ?it/s]


torch.Size([2, 184, 630, 630])

In [4]:
for data in tqdm(train_loader):
    exp_names = data["exp_name"]
    normalized_tomogram = data["normalized_tomogram"]
    segmentation_map = data["segmentation_map"]
    break

  0%|          | 0/12 [00:04<?, ?it/s]


In [5]:
exp_names, normalized_tomogram.shape, segmentation_map.shape

(['TS_73_6', 'TS_6_6'],
 torch.Size([2, 184, 630, 630]),
 torch.Size([2, 184, 630, 630]))

In [6]:
import cc3d

all_pred = []

for data in tqdm(train_dataset):
    exp_name = data["exp_name"]
    normalized_tomogram = data["normalized_tomogram"]
    segmentation_map = data["segmentation_map"]
    # print(segmentation_map.shape)
    gt = segmentation_map  # .numpy()

    cls_pos = []
    Ascale_pos = []
    res2ratio = CFG.resolution2ratio
    # exp_name = exp_names[i]

    for pred_cls in range(1, len(CFG.particles_name) + 1):
        cc, P = cc3d.connected_components(gt == pred_cls, return_N=True)
        stats = cc3d.statistics(cc)

        for z, y, x in stats["centroids"][1:]:
            Ascale_z = z * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_x = x * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_y = y * res2ratio[CFG.resolution] / res2ratio["A"]

            cls_pos.append([pred_cls, z, y, x])
            Ascale_pos.append([pred_cls, Ascale_z, Ascale_y, Ascale_x])

    pred_original_df = create_df(Ascale_pos, exp_name)
    all_pred.append(pred_original_df)

all_pred = pd.concat(all_pred).reset_index().drop_duplicates(subset=["x", "y", "z"])

100%|██████████| 24/24 [00:49<00:00,  2.04s/it]


In [7]:
def create_gt_df(base_dir, exp_names):
    result_df = None
    particle_names = CFG.particles_name

    for exp_name in exp_names:
        for particle in particle_names:
            np_corrds = read_info_json(
                base_dir=base_dir, exp_name=exp_name, particle_name=particle
            )  # (n, 3)
            # 各行にexp_nameとparticle_name追加
            particle_df = pd.DataFrame(np_corrds, columns=["z", "y", "x"])
            particle_df["experiment"] = exp_name
            particle_df["particle_type"] = particle

            if result_df is None:
                result_df = particle_df
            else:
                result_df = pd.concat([result_df, particle_df], axis=0).reset_index(
                    drop=True
                )

    result_df = result_df.reset_index()
    result_df = result_df[["index", "experiment", "particle_type", "x", "y", "z"]]

    return result_df


gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", CFG.train_exp_names)
gt_df

,index,experiment,particle_type,x,y,z
0,0,TS_73_6,apo-ferritin,268.662,4730.318,916.115
1,1,TS_73_6,apo-ferritin,238.946,4853.061,909.898
2,2,TS_73_6,apo-ferritin,83.114,5729.560,1219.524
3,3,TS_73_6,apo-ferritin,582.143,2769.968,1076.364
4,4,TS_73_6,apo-ferritin,510.389,2157.244,362.438
...,...,...,...,...,...,...
1124,1124,TS_6_6,virus-like-particle,2609.876,4569.876,1169.759
1125,1125,TS_6_6,virus-like-particle,2213.287,4135.017,1286.851
1126,1126,TS_6_6,virus-like-particle,3303.905,5697.825,789.744
1127,1127,TS_6_6,virus-like-particle,1008.748,5949.213,1077.303


In [8]:
score(all_pred, gt_df, row_id_column_name="index", distance_multiplier=0.5, beta=4)

0.987299215367147

In [9]:
all_pred

,index,experiment,particle_type,x,y,z
0,0,TS_73_6,apo-ferritin,5395.0,4745.0,35.0
1,1,TS_73_6,apo-ferritin,5265.0,4805.0,35.0
2,2,TS_73_6,apo-ferritin,5535.0,3525.0,45.0
3,3,TS_73_6,apo-ferritin,5415.0,4625.0,45.0
4,4,TS_73_6,apo-ferritin,4635.0,2185.0,55.0
...,...,...,...,...,...,...
3963,135,TS_6_6,virus-like-particle,395.0,2705.0,965.0
3964,136,TS_6_6,virus-like-particle,1005.0,5945.0,1075.0
3965,137,TS_6_6,virus-like-particle,2605.0,4565.0,1165.0
3966,138,TS_6_6,virus-like-particle,3545.0,995.0,1195.0


In [10]:
encoder = timm.create_model(
    model_name=CFG.model_name,
    pretrained=True,
    in_chans=3,
    num_classes=0,
    global_pool="",
    features_only=True,
)
model = Unet3D(encoder=encoder).to("cuda")
model.load_state_dict(torch.load("./best_model.pth"))

<All keys matched successfully>

In [11]:
from tqdm import tqdm

# ############### validation ################
train_nshuffle_original_tomogram = defaultdict(list)
train_nshuffle_pred_tomogram = defaultdict(list)
train_nshuffle_gt_tomogram = defaultdict(list)
train_cls_pos = defaultdict(list)
train_cls_Apos = defaultdict(list)

valid_original_tomogram = defaultdict(list)
valid_pred_tomogram = defaultdict(list)
valid_gt_tomogram = defaultdict(list)
valid_cls_pos = defaultdict(list)
valid_cls_Apos = defaultdict(list)

train_mean_scores = []
valid_mean_scores = []

# # for exp_name in tqdm(CFG.train_exp_names):
# for exp_name in tqdm(CFG.train_exp_names[:5]):  # 5つのデータで試す
#     inferenced_array, n_tomogram, segmentation_map = inference(
#         model, exp_name, train=True
#     )
#     train_nshuffle_pred_tomogram[exp_name] = inferenced_array
#     train_nshuffle_gt_tomogram[exp_name] = segmentation_map.squeeze(0)
#     train_nshuffle_original_tomogram[exp_name] = n_tomogram.squeeze(0)

#     mean_score, scores, pred_df, gt_df, pred_cls_pos, pred_Ascale_pos = (
#         visualize_epoch_results(
#             train_nshuffle_pred_tomogram,
#             base_dir="../../inputs/train/overlay/ExperimentRuns/",
#             sikii_dict=CFG.initial_sikii,
#         )
#     )
#     train_cls_pos[exp_name] = pred_cls_pos
#     train_cls_Apos[exp_name] = pred_Ascale_pos
#     train_mean_scores.append(mean_score)
# print("train_mean_scores", np.mean(train_mean_scores))

for exp_name in tqdm(CFG.valid_exp_names):
    inferenced_array, n_tomogram, segmentation_map = inference(
        model, exp_name, train=True
    )
    valid_pred_tomogram[exp_name] = inferenced_array
    valid_gt_tomogram[exp_name] = segmentation_map.squeeze(0)
    valid_original_tomogram[exp_name] = n_tomogram.squeeze(0)

    mean_score, scores, pred_df, gt_df, pred_cls_pos, pred_Ascale_pos = (
        visualize_epoch_results(
            valid_pred_tomogram,
            base_dir="../../inputs/train/overlay/ExperimentRuns/",
            sikii_dict=CFG.initial_sikii,
        )
    )
    valid_cls_pos[exp_name] = pred_cls_pos
    valid_cls_Apos[exp_name] = pred_Ascale_pos
    valid_mean_scores.append(mean_score)
print("valid_mean_scores", np.mean(valid_mean_scores))

100%|██████████| 1/1 [00:31<00:00, 31.87s/it]

valid_mean_scores 0.0


In [12]:
inferenced_array, n_tomogram, segmentation_map = inference(model, exp_name, train=True)
pred_original_df = inference2pos(
    pred_segmask=inferenced_array,
    exp_name=exp_name,
    sikii_dict=CFG.initial_sikii,
)
pred_df

,index,experiment,particle_type,x,y,z
0,0,TS_5_4,apo-ferritin,3145.0,3145.0,35.000000
1,1,TS_5_4,apo-ferritin,3145.0,3145.0,0.000000
2,6,TS_5_4,ribosome,3145.0,3145.0,34.999962
3,7,TS_5_4,ribosome,3145.0,3145.0,0.000227
4,8,TS_5_4,thyroglobulin,3145.0,3145.0,34.999824
5,9,TS_5_4,thyroglobulin,3145.0,3145.0,0.001058
6,10,TS_5_4,virus-like-particle,3145.0,3145.0,34.999765
7,11,TS_5_4,virus-like-particle,3145.0,3145.0,0.001411


In [15]:
def inference2pos(pred_segmask, exp_name, sikii_dict):
    import cc3d

    cls_pos = []
    Ascale_pos = []
    res2ratio = CFG.resolution2ratio

    for pred_cls in range(1, len(CFG.particles_name) + 1):
        sikii = sikii_dict[CFG.cls2particles[pred_cls]]
        # print(pred_segmask[pred_cls].shape)
        cc, P = cc3d.connected_components(pred_segmask[pred_cls] > sikii, return_N=True)
        # cc, P = cc3d.connected_components(pred_segmask == pred_cls, return_N=True)
        stats = cc3d.statistics(cc)

        for z, y, x in stats["centroids"][1:]:
            Ascale_z = z * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_x = x * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_y = y * res2ratio[CFG.resolution] / res2ratio["A"]

            cls_pos.append([pred_cls, z, y, x])
            Ascale_pos.append([pred_cls, Ascale_z, Ascale_y, Ascale_x])

    pred_original_df = create_df(Ascale_pos, exp_name)

    return pred_original_df

In [16]:
pred_original_df

,experiment,particle_type,x,y,z
0,TS_5_4,apo-ferritin,5868.557692,5130.192308,68.413462
1,TS_5_4,apo-ferritin,5739.484536,5100.206186,80.618557
2,TS_5_4,apo-ferritin,5711.000000,5000.600000,117.800000
3,TS_5_4,apo-ferritin,2795.634921,1630.992063,195.039683
4,TS_5_4,apo-ferritin,2392.077922,1698.311688,198.441558
...,...,...,...,...,...
102,TS_5_4,virus-like-particle,2616.896285,3742.167183,854.791022
103,TS_5_4,virus-like-particle,112.446256,5393.632320,916.582654
104,TS_5_4,virus-like-particle,2628.217716,4215.357524,974.909285
105,TS_5_4,virus-like-particle,3440.740580,6169.068861,1027.804244


In [17]:
score(pred_df, gt_df, row_id_column_name="index", distance_multiplier=1.0, beta=4)

0.0

In [18]:
pred_df[pred_df["particle_type"] == "apo-ferritin"].sort_values("z")

,index,experiment,particle_type,x,y,z
1,1,TS_5_4,apo-ferritin,3145.0,3145.0,0.0
0,0,TS_5_4,apo-ferritin,3145.0,3145.0,35.0


In [19]:
gt_df[gt_df["particle_type"] == "apo-ferritin"].sort_values("z")

,index,experiment,particle_type,x,y,z
17,17,TS_5_4,apo-ferritin,5870.268,5131.104,76.321
45,45,TS_5_4,apo-ferritin,5748.253,5115.846,108.033
44,44,TS_5_4,apo-ferritin,5715.014,4998.216,115.143
26,26,TS_5_4,apo-ferritin,2801.725,1630.415,204.473
22,22,TS_5_4,apo-ferritin,4829.392,4473.384,208.707
29,29,TS_5_4,apo-ferritin,2626.828,2295.534,254.337
27,27,TS_5_4,apo-ferritin,2504.120,2311.852,267.454
28,28,TS_5_4,apo-ferritin,2499.282,2197.394,298.989
39,39,TS_5_4,apo-ferritin,2340.306,2283.227,345.429
8,8,TS_5_4,apo-ferritin,1072.449,2963.265,384.014


In [20]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

num_classes = len(CFG.particles_name)  # クラス数
colors = plt.cm.tab10(
    np.arange(len(CFG.particles_name))
)  # "tab10" カラーマップから色を取得

# ListedColormap を作成
class_colormap = ListedColormap(colors)


def plot_with_colormap(data, title, original_tomogram):
    masked_data = np.ma.masked_where(data <= 0, data)  # クラス0をマスク
    plt.imshow(original_tomogram, cmap="gray")
    im = plt.imshow(masked_data, cmap=class_colormap)
    plt.title(title)
    plt.axis("off")
    return im


def imshow_result(pred, gt, original, index):
    # plt.figure(figsize=(20, 5))
    ax = plt.subplot(1, 3, 1)
    plot_with_colormap(
        pred[index],
        "Train-Prediction",
        original[index],
    )
    ax = plt.subplot(1, 3, 2)
    plot_with_colormap(gt[index], "Gt", original[index])

    ax = plt.subplot(1, 3, 3)
    plt.imshow(original[index], cmap="gray")
    plt.axis("off")

    plt.show()

In [21]:
# exp_name = "TS_5_4"
# index = 12
# pred = train_nshuffle_pred_tomogram[exp_name].argmax(0)  # (92, 315, 315)
# gt = train_nshuffle_gt_tomogram[exp_name]
# original = train_nshuffle_original_tomogram[exp_name]

# # imshow_result(pred, gt, original, index)

# for i in range(42):
#     imshow_result(pred, gt, original, i)

In [22]:
# exp_name = "TS_5_4"
# index = 12
# pred = train_nshuffle_pred_tomogram[exp_name].argmax(0)  # (92, 315, 315)
# gt = train_nshuffle_gt_tomogram[exp_name]
# original = train_nshuffle_original_tomogram[exp_name]

# # imshow_result(pred, gt, original, index)

# for i in range(42):
#     imshow_result(pred, gt, original, i)

In [23]:
# exp_name = CFG.valid_exp_names[-1]

# pred = valid_pred_tomogram[exp_name].argmax(0)
# gt = valid_gt_tomogram[exp_name]
# original = valid_original_tomogram[exp_name]

# for i in range(42):
#     imshow_result(pred, gt, original, i)

In [24]:
# exp_name = CFG.train_exp_names[-1]

# pred_cls_pos = train_cls_pos[exp_name]

# exp_name, np.array(pred_cls_pos).max(axis=0)

In [25]:
# exp_name = CFG.valid_exp_names[0]

# pred_cls_pos = valid_cls_pos[exp_name]

# np.array(pred_cls_pos).max(axis=0)

In [26]:
gt_df

,index,experiment,particle_type,x,y,z
0,0,TS_5_4,apo-ferritin,468.514,5915.906,604.167
1,1,TS_5_4,apo-ferritin,5674.694,1114.354,565.068
2,2,TS_5_4,apo-ferritin,5744.509,1049.172,653.712
3,3,TS_5_4,apo-ferritin,5880.769,1125.348,579.560
4,4,TS_5_4,apo-ferritin,4661.667,1269.497,810.409
...,...,...,...,...,...,...
135,135,TS_5_4,virus-like-particle,2636.539,4214.980,965.410
136,136,TS_5_4,virus-like-particle,3137.396,3572.460,372.914
137,137,TS_5_4,virus-like-particle,3294.133,3027.464,674.070
138,138,TS_5_4,virus-like-particle,2997.686,4948.218,1169.375


In [27]:
score(pred_df, gt_df, row_id_column_name="index", distance_multiplier=1.0, beta=4)

0.0

In [28]:
def create_gt_df(base_dir, exp_names):
    result_df = None
    particle_names = CFG.particles_name

    for exp_name in exp_names:
        for particle in particle_names:
            np_corrds = read_info_json(
                base_dir=base_dir, exp_name=exp_name, particle_name=particle
            )  # (n, 3)
            # 各行にexp_nameとparticle_name追加
            particle_df = pd.DataFrame(np_corrds, columns=["z", "y", "x"])
            particle_df["experiment"] = exp_name
            particle_df["particle_type"] = particle

            if result_df is None:
                result_df = particle_df
            else:
                result_df = pd.concat([result_df, particle_df], axis=0).reset_index(
                    drop=True
                )

    result_df = result_df.reset_index()
    result_df = result_df[["index", "experiment", "particle_type", "x", "y", "z"]]

    return result_df

In [29]:
# exp_name = CFG.valid_exp_names[0]
# pred = valid_pred_tomogram[exp_name].argmax(0)
# gt = valid_gt_tomogram[exp_name]
# original = valid_original_tomogram[exp_name]
import timm

encoder = timm.create_model(
    model_name=CFG.model_name,
    pretrained=True,
    in_chans=3,
    num_classes=0,
    global_pool="",
    features_only=True,
)
model = Unet3D(encoder=encoder).to("cuda")
model.load_state_dict(torch.load("./best_model.pth"))

exp_name = CFG.train_exp_names[2]
pred = train_nshuffle_pred_tomogram[exp_name].argmax(0)
gt = train_nshuffle_gt_tomogram[exp_name]
original = train_nshuffle_original_tomogram[exp_name]

base_dir = "../../inputs/train/overlay/ExperimentRuns/"
gt_df = create_gt_df(base_dir=base_dir, exp_names=[exp_name])

import cc3d

cls_pos = []
Ascale_pos = []
res2ratio = CFG.resolution2ratio

for pred_cls in range(1, len(CFG.particles_name) + 1):
    print(pred_cls, CFG.cls2particles[pred_cls])
    cc, P = cc3d.connected_components(pred == pred_cls, return_N=True)
    stats = cc3d.statistics(cc)

    for z, y, x in stats["centroids"]:
        Ascale_z = z * res2ratio[CFG.resolution] / res2ratio["A"]
        Ascale_x = x * res2ratio[CFG.resolution] / res2ratio["A"]
        Ascale_y = y * res2ratio[CFG.resolution] / res2ratio["A"]

        cls_pos.append([pred_cls, z, y, x])
        Ascale_pos.append([pred_cls, Ascale_z, Ascale_y, Ascale_x])

pred_original_df = create_df(Ascale_pos, exp_name).drop_duplicates(
    subset=["x", "y", "z"]
)

score(
    pred_original_df, gt_df, row_id_column_name="index", distance_multiplier=1.0, beta=4
)

AttributeError: 'list' object has no attribute 'argmax'

In [ ]:
import timm

encoder = timm.create_model(
    model_name=CFG.model_name,
    pretrained=True,
    in_chans=3,
    num_classes=0,
    global_pool="",
    features_only=True,
)
model = Unet3D(encoder=encoder).to("cuda")
model.load_state_dict(torch.load("./best_model.pth"))

exp_name = CFG.valid_exp_names[-1]
pred = valid_pred_tomogram[exp_name]
gt = valid_gt_tomogram[exp_name]
original = valid_original_tomogram[exp_name]

base_dir = "../../inputs/train/overlay/ExperimentRuns/"
gt_df = create_gt_df(base_dir=base_dir, exp_names=[exp_name])


for constant in np.linspace(0.2, 0.9, 20):
    initial_sikii = {
        "apo-ferritin": constant,
        "beta-amylase": constant,
        "beta-galactosidase": constant,
        "ribosome": constant,
        "thyroglobulin": constant,
        "virus-like-particle": constant,
    }

    import cc3d

    cls_pos = []
    Ascale_pos = []
    res2ratio = CFG.resolution2ratio

    for pred_cls in range(1, len(CFG.particles_name) + 1):
        sikii = initial_sikii[CFG.cls2particles[pred_cls]]
        cc, P = cc3d.connected_components(pred[pred_cls] > sikii, return_N=True)
        stats = cc3d.statistics(cc)

        for z, y, x in stats["centroids"]:
            Ascale_z = z * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_x = x * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_y = y * res2ratio[CFG.resolution] / res2ratio["A"]

            cls_pos.append([pred_cls, z, y, x])
            Ascale_pos.append([pred_cls, Ascale_z, Ascale_y, Ascale_x])

    pred_original_df = create_df(Ascale_pos, exp_name).drop_duplicates(
        subset=["x", "y", "z"]
    )

    score_ = score(
        pred_original_df,
        gt_df,
        row_id_column_name="index",
        distance_multiplier=1.0,
        beta=4,
    )

    print(sikii, score_)

In [34]:
import timm

encoder = timm.create_model(
    model_name=CFG.model_name,
    pretrained=True,
    in_chans=3,
    num_classes=0,
    global_pool="",
    features_only=True,
)
model = Unet3D(encoder=encoder).to("cuda")
model.load_state_dict(torch.load("./best_model.pth"))

exp_name = CFG.valid_exp_names[0]
pred = valid_pred_tomogram[exp_name]
gt = valid_gt_tomogram[exp_name]
original = valid_original_tomogram[exp_name]

base_dir = "../../inputs/train/overlay/ExperimentRuns/"
gt_df = create_gt_df(base_dir=base_dir, exp_names=[exp_name])


for constant in np.linspace(0.2, 0.9, 20):
    initial_sikii = {
        "apo-ferritin": constant,
        "beta-amylase": constant,
        "beta-galactosidase": constant,
        "ribosome": constant,
        "thyroglobulin": constant,
        "virus-like-particle": constant,
    }

    import cc3d

    cls_pos = []
    Ascale_pos = []
    res2ratio = CFG.resolution2ratio

    for pred_cls in range(1, len(CFG.particles_name) + 1):
        sikii = initial_sikii[CFG.cls2particles[pred_cls]]
        cc, P = cc3d.connected_components(pred[pred_cls] > sikii, return_N=True)
        stats = cc3d.statistics(cc)

        for z, y, x in stats["centroids"]:
            Ascale_z = z * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_x = x * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_y = y * res2ratio[CFG.resolution] / res2ratio["A"]

            cls_pos.append([pred_cls, z, y, x])
            Ascale_pos.append([pred_cls, Ascale_z, Ascale_y, Ascale_x])

    pred_original_df = create_df(Ascale_pos, exp_name).drop_duplicates(
        subset=["x", "y", "z"]
    )

    score_ = score(
        pred_original_df,
        gt_df,
        row_id_column_name="index",
        distance_multiplier=1.0,
        beta=4,
    )

    print(sikii, score_)

NameError: name 'valid_pred_tomogram' is not defined

In [35]:
encoder = timm.create_model(
    model_name=CFG.model_name,
    pretrained=True,
    in_chans=3,
    num_classes=0,
    global_pool="",
    features_only=True,
)
model = Unet3D(encoder=encoder).to("cuda")
model.load_state_dict(torch.load("./best_model.pth"))

exp_name = CFG.valid_exp_names[0]
inferenced_array, n_tomogram, segmentation_map = inference(model, exp_name, train=True)

0.7303962244998289

In [37]:
for constant in np.linspace(0.2, 0.9, 20):
    initial_sikii = {
        "apo-ferritin": constant,
        "beta-amylase": constant,
        "beta-galactosidase": constant,
        "ribosome": constant,
        "thyroglobulin": constant,
        "virus-like-particle": constant,
    }

    pred_original_df = inference2pos(
        pred_segmask=inferenced_array,
        exp_name=exp_name,
        sikii_dict=initial_sikii,
    )
    gt_df = create_gt_df(
        base_dir="../../inputs/train/overlay/ExperimentRuns/", exp_names=[exp_name]
    )

    s = score(
        pred_original_df,
        gt_df,
        row_id_column_name="index",
        distance_multiplier=1.0,
        beta=4,
    )
    print(constant, s)

0.2 0.50816555053324
0.2368421052631579 0.5459167465403733
0.2736842105263158 0.5511186139902575
0.31052631578947365 0.5944276213159514
0.34736842105263155 0.6128575529806081
0.38421052631578945 0.6919808803664267
0.42105263157894735 0.6544020391846213
0.45789473684210524 0.6977925081331098
0.49473684210526314 0.7308285321620421
0.531578947368421 0.7234145272650483
0.5684210526315789 0.5698279025155875
0.6052631578947368 0.5449937091169922
0.6421052631578947 0.537768200339582
0.6789473684210525 0.5445142100746757
0.7157894736842105 0.5874605333623334
0.7526315789473683 0.47301704906095426
0.7894736842105263 0.4960349809187644
0.8263157894736841 0.3385308640653399
0.8631578947368421 0.14416177429876062
0.9 0.13821138211382114


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion
import matplotlib.pyplot as pp


def detect_peaks(image, cls_index):
    """
    Takes an image and detect the peaks usingthe local maximum filter.
    Returns a boolean mask of the peaks (i.e. 1 when
    the pixel's value is the neighborhood maximum, 0 otherwise)
    """

    # define an 8-connected neighborhood
    neighborhood = generate_binary_structure(2, 2)

    # apply the local maximum filter; all pixel of maximal value
    # in their neighborhood are set to 1
    local_max = maximum_filter(image, footprint=neighborhood) == image
    # local_max is a mask that contains the peaks we are
    # looking for, but also the background.
    # In order to isolate the peaks we must remove the background from the mask.

    # we create the mask of the background
    background = image == 0

    # a little technicality: we must erode the background in order to
    # successfully subtract it form local_max, otherwise a line will
    # appear along the background border (artifact of the local maximum filter)
    eroded_background = binary_erosion(
        background, structure=neighborhood, border_value=cls_index
    )

    # we obtain the final mask, containing only peaks,
    # by removing the background from the local_max mask (xor operation)
    detected_peaks = local_max ^ eroded_background

    return detected_peaks


# test
x = np.linspace(0, 4 * np.pi, 100)
y = np.sin(x).reshape(-1, 10)
# peaks = detect_peaks(y)

In [67]:
import numpy as np
from scipy.ndimage import (
    binary_dilation,
    binary_erosion,
    binary_opening,
    binary_closing,
)


def apply_morphology(
    segmentation: np.ndarray, class_range: tuple[int, int]
) -> np.ndarray:
    """
    セマンティックセグメンテーション結果にモルフォロジー処理を適用して鮮鋭化します。

    Args:
        segmentation (np.ndarray): セグメンテーション結果 (2Dまたは3D配列)。
        class_range (tuple[int, int]): モルフォロジー処理対象のクラス範囲 (min_class, max_class)。

    Returns:
        np.ndarray: モルフォロジー処理後のセグメンテーション結果。
    """
    # 背景クラス (0) は処理対象外
    processed_segmentation = np.zeros_like(segmentation)
    for cls in range(class_range[0], class_range[1] + 1):
        # 特定のクラスのバイナリマスクを作成
        binary_mask = segmentation == cls

        # モルフォロジー処理 (例: 開操作)
        refined_mask = binary_opening(
            binary_mask, structure=np.ones((3, 3))
        )  # 3x3の構造要素を使用
        refined_mask = binary_closing(
            refined_mask, structure=np.ones((3, 3))
        )  # 閉操作でギャップを埋める

        # 処理後のマスクを反映
        processed_segmentation[refined_mask] = cls

    return processed_segmentation


# 使用例
# セグメンテーション結果 (例)
segmentation_result = np.random.randint(
    0, 8, size=(100, 100)
)  # ランダムなセグメンテーション結果

# クラス1~7を鮮鋭化
processed_result = apply_morphology(segmentation_result, class_range=(1, 7))

In [89]:
import numpy as np
from scipy.ndimage import binary_opening, binary_closing, label


def refine_segmentation(
    segmentation: np.ndarray, class_range: tuple[int, int], min_size: int = 10
) -> np.ndarray:
    """
    小さなクラス同士を分離したまま、セマンティックセグメンテーション結果を鮮鋭化します。

    Args:
        segmentation (np.ndarray): セグメンテーション結果 (2Dまたは3D配列)。
        class_range (tuple[int, int]): 処理対象のクラス範囲 (min_class, max_class)。
        min_size (int): 小さい領域を除去する際の最小ピクセルサイズ。

    Returns:
        np.ndarray: 鮮鋭化されたセグメンテーション結果。
    """
    processed_segmentation = np.zeros_like(segmentation)

    for cls in range(class_range[0], class_range[1] + 1):
        # 特定クラスのバイナリマスクを作成
        binary_mask = segmentation == cls

        # 開操作でノイズ除去、閉操作でギャップを埋める
        refined_mask = binary_opening(binary_mask, structure=np.ones((1, 1)))
        refined_mask = binary_closing(refined_mask, structure=np.ones((1, 1)))

        # ラベリングで分離された領域を管理
        labeled_mask, num_features = label(refined_mask)

        # 小さい領域をフィルタリング
        for region in range(1, num_features + 1):
            region_mask = labeled_mask == region
            if np.sum(region_mask) >= min_size:  # 最小サイズ以上の領域を保持
                processed_segmentation[region_mask] = cls

    return processed_segmentation

In [90]:
seg = inferenced_array.argmax(0)
zeros_array = np.zeros_like(seg)

In [98]:
for i in range(zeros_array.shape[0]):
    zeros_array[i] = refine_segmentation(seg[i], class_range=(1, 6), min_size=3)

In [ ]:
i = 45
plt.figure(figsize=(15, 15))
ax = plt.subplot(1, 2, 1)
plt.imshow(inferenced_array.argmax(0)[i])

ax = plt.subplot(1, 2, 2)
plt.imshow(zeros_array[45])